In [167]:
import requests
import re
import time
import json
from tqdm import tqdm
import psycopg2

In [168]:
book_name = 'Tusculan Disputations - Book 1'
page_name = 'Paragraph 18'
url_source = 'texts/tusculan_disputations/book_1/paragraph_18.txt'

In [169]:
conn = psycopg2.connect(
    host='localhost',
    database='tollelege',
    user='tollelege',
    password='1$TolleLegge$1'
)
cursor = conn.cursor()
cursor.execute(f'''select p.Id, b.title, p.title from reader_page p, reader_book b where p.book_id = b.Id
               and b.title = '{book_name}'
               and p.title = '{page_name}'
               ''')
page_pk = cursor.fetchall()[0][0]
    

In [170]:
def parse_text(text):
    lines = []
    text = re.split('\.|\?|\!|\:|\;|\,', text)
    
    for line in text:
        line = line.split(' ')
        line = [word for word in line if word != '']
        lines.append(line)

    lines.remove([])
    return lines

In [171]:
def get_punct_marks(text):
    punct_marks = [char for char in text if char not in 'abcdefghijklmnopqurstuvwxyzABCDEFGHIJKLMNOPQURSTUVWXYZ[]()123456789']
    punct_marks = [p for p in punct_marks if p != ' ']
    return punct_marks

In [172]:
def get_whitakers_data(word):
    r = requests.get(f'https://latin-words.com/cgi-bin/translate.cgi?query={word}')
    parsed_response = r.json()['message'].strip()
    time.sleep(.1)
    return parsed_response

In [173]:
with open(url_source, 'r') as f:
    text_data = f.read()

In [174]:
parsed_text = parse_text(text_data)
punct_marks = get_punct_marks(text_data)

In [175]:
morphemes = []

page_index = 1
counter = 1

for line, punct in tqdm(zip(parsed_text, punct_marks)):
    for word in line:
        word = word.translate({ord(x): None for x in "[]()123456789"})
        if word != "":
            morpheme_data = (
                word,
                page_index,
                get_whitakers_data(word),
                page_pk
            )
            morphemes.append(morpheme_data)
            counter += 1
            page_index += 1

    punct_data = (
        punct,
        page_index,
        "Punctuation (no data)",
        page_pk
    )

    morphemes.append(punct_data)
    counter += 1
    page_index += 1


21it [01:02,  3.00s/it]


In [176]:
morphemes

[('Mors',
  1,
  'mors                 N      3 3 NOM S F                 \nmors                 N      3 3 VOC S F                 \nmors, mortis  N (3rd) F   [XXXAX]  \ndeath; corpse; annihilation;',
  15),
 ('igitur',
  2,
  'igitur               CONJ                               \nigitur  CONJ   [XXXAO]  \ntherefore (postpositive), so/then; consequently; accordingly; well/in that case',
  15),
 ('ipsa',
  3,
  'ips.a                PRON   6 2 NOM S F                 \nips.a                PRON   6 2 NOM P N                 \nips.a                PRON   6 2 ACC P N                 \nipse, ipsa, ipsum  PRON   [XXXAO]  \nhimself/herself/itself; the very/real/actual one; in person; themselves (pl.);',
  15),
 (',', 4, 'Punctuation (no data)', 15),
 ('quae',
  5,
  'qu.ae                PRON   1 0 NOM P F                 \n [XXXAO]  \nwho; that; which, what; of which kind/degree; person/thing/time/point that;\r\nwho/what/which?, what/which one/man/person/thing? what kind/type of?;\r\nw

In [177]:
for morpheme in morphemes:
    cursor.execute(
        '''
        insert into reader_morpheme (morpheme, page_index, parse, page_id)
        values (%s, %s, %s, %s)
        on conflict (page_index, page_id)
        do
        update set morpheme = EXCLUDED.morpheme, parse = EXCLUDED.parse
        ''',
        morpheme
    )

In [178]:
conn.commit()
conn.close()